V6

In [4]:
import pandas as pd
from a_PyCaller import process_urls
from tqdm import tqdm

def process_and_print_results(url, all_pole_studio_data, all_workshops_data, all_workshop_details_data):
    data = process_urls([url])

    if data:
        for key, df in data.items():
            if df is not None and not df.empty:
                print(f"{key.replace('_', ' ').title()}: {len(df)} entries")

                # Update the appropriate DataFrame
                if key == 'pole_studio_data':
                    all_pole_studio_data = pd.concat([all_pole_studio_data, df], ignore_index=True)
                elif key == 'workshops_data':
                    all_workshops_data = pd.concat([all_workshops_data, df], ignore_index=True)
                elif key == 'workshop_details':
                    all_workshop_details_data = pd.concat([all_workshop_details_data, df], ignore_index=True)

    return all_pole_studio_data, all_workshops_data, all_workshop_details_data

def main():
    # Load initial URLs
    all_found_urls_s = pd.read_csv("your_output_file.csv")
    initial_urls = list(set(all_found_urls_s["0"])) # Delete "[:3]" to scrape all Urls

    # Initialize DataFrames
    all_pole_studio_data = pd.DataFrame()
    all_workshops_data = pd.DataFrame()
    all_workshop_details_data = pd.DataFrame()

    # Process each URL with tqdm
    with tqdm(total=len(initial_urls), desc="Processing URLs", dynamic_ncols=True) as pbar:
        for url in initial_urls:
            all_pole_studio_data, all_workshops_data, all_workshop_details_data = process_and_print_results(
                url, all_pole_studio_data, all_workshops_data, all_workshop_details_data
            )
            pbar.update(1)
            pbar.set_postfix_str(f"Current URL: {url}", refresh=True)

    # # Export DataFrames to CSV files
    all_pole_studio_data.to_csv("Pole_Studio_Übersicht_S.csv", index=False)
    all_workshops_data.to_csv("Workshop_Liste_SW.csv", index=False)
    all_workshop_details_data.to_csv("Workshop_Übersicht_E.csv", index=False)

    # Return the final DataFrames
    return all_pole_studio_data, all_workshops_data, all_workshop_details_data

# Run the main function and get the final DataFrames
result_pole_studio, result_workshops, result_workshop_details = main()

# Now you can access result_pole_studio, result_workshops, and result_workshop_details outside the function
# print(result_pole_studio)
# print(result_workshops)
# print(result_workshop_details)


Processing URLs:   0%|          | 0/337 [00:00<?, ?it/s]

Starting URL reconstruction...


Processing URLs:   0%|          | 1/337 [00:01<05:49,  1.04s/it, Current URL: https://www.eversports.de/s/fl'air-studios-berlin]

Error during HTTP request: 404 Client Error: Not Found for url: https://www.eversports.de/s/fl'air-studios-berlin
Converting to DataFrame...
Validating URLs...
Processing completed.
Starting URL reconstruction...


2024-02-03 16:26:53,771 - INFO - Processing URL 1/4: https://www.eversports.de/s/pole-faction-darmstadt


...

Converting to DataFrame...
Validating URLs...


2024-02-03 16:57:56,233 - INFO - URL https://www.eversports.de/s/welovepoledance is valid.


Scraping Pole Studio Data from https://www.eversports.de/s/welovepoledance...


Processing URLs: 100%|██████████| 337/337 [31:05<00:00,  5.54s/it, Current URL: https://www.eversports.de/s/welovepoledance] 

Processing completed.
Pole Studio Data: 1 entries


In [5]:
result_pole_studio

,PoleStudio_Name,Adresse,PLZ,Stadt,Straße,Buttons,Pole Studio Beschreibung,E-Mail,Homepage,Telefon,URL_S,Art,Angebot,Created Date,Updated Date
0,Pole Faction - Darmstadt,"[Heidelberger Landstrasse , 64297 Darmstadt]",64297,Darmstadt,Heidelberger Landstrasse,"[Übersicht, Klassen, Workshops, Preise, Team]",Wir sind ein Pole Dance Studio mit einem gemis...,info@pole-faction.de,https://www.pole-faction.de,+49 15225933500,https://www.eversports.de/s/pole-faction-darms...,"[Poledance, Aerial Hoop, Aerial Hoop, Poledanc...",2 Angebote für Neukund:innen,2024-02-03 16:26:59,2024-02-03 16:26:59
1,Pole Dance Palazzo Sportivo,"[Rudolf-Diesel-Straße 54 , 69190 Walldorf]",69190,Walldorf,Rudolf-Diesel-Straße 54,[Übersicht],Herzlich Willkommen zum Online Buchungssystem ...,pole@palazzo-sportivo.de,http://www.poledance-rheinneckar.de,+49 15678684972,https://www.eversports.de/s/pole-dance-palazzo...,"[16 Rezensionen, Aerial Hoop, Aerial Hammock, ...",1 Angebot für Neukund:innen,2024-02-03 16:27:05,2024-02-03 16:27:05
2,Pole Statement,"[Siemensstraße 40 , 69123 Heidelberg]",69123,Heidelberg,Siemensstraße 40,"[Übersicht, Klassen, Kurse, Events, Preise, Team]",Bei Pole Statement unterrichten wir Pole Dance...,info@polestatement.de,http://www.polestatement.de,+49 17672506814,https://www.eversports.de/s/pole-statement,"[Poledance, Outdoor Fitness, Poledance, Outdoo...",2 Angebote für Neukund:innen,2024-02-03 16:27:15,2024-02-03 16:27:15
3,ArtLetik UG Laufen und Meer,"[Arentestraße 15, 26607 Aurich]",26607,Aurich,Arentestraße 15,[Übersicht],Unter der Leitung von Anja Musiol werden profe...,office@artletik.de,https://www.artletik.de/,+49 1705837103,https://www.eversports.de/s/artletik-ug-laufen...,"[(Noch keine Bewertungen), Fitness]",None,2024-02-03 16:27:19,2024-02-03 16:27:19
4,Les.Pole,"[Bellingerstraße 12 , 36043 Fulda]",36043,Fulda,Bellingerstraße 12,"[Übersicht, Klassen, Workshops, Preise, Team]","Poledance, Chairdance, Exotic, Floorwork.........",info@les-pole.de,https://www.instagram.com/les.pole,+49 15730850054,https://www.eversports.de/s/lespole,"[Poledance, Power Pole, Poledance, Poledance, ...",1 Angebot für Neukund:innen,2024-02-03 16:27:29,2024-02-03 16:27:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Anni's Pole-Dance Neumünster,"[Kieler Straße 52 , 24534 Neumünster]",24534,Neumünster,Kieler Straße 52,"[Übersicht, Klassen, Trainings, Kurse, Team]","Hallo lieber Schüler,schön, dass du zu mir gef...",anni@annis-poledance.de,https://www.annis-poledance.de/,+49 15253821082,https://www.eversports.de/s/annis-pole-dance-n...,"[Poledance, Handstand, Poledance, Aerial Hoop,...",Zum Stundenplan,2024-02-03 16:57:22,2024-02-03 16:57:22
156,UpsideDown Studio Friedrichsdorf,"[Am Houiller Platz 4D , 61381 Friedrichsdorf]",61381,Friedrichsdorf,Am Houiller Platz 4D,"[Übersicht, Klassen, Kurse, Preise, Team]",Das Studio UpsideDown ist in Friedrichsdorf zu...,info@upsidedown-studio.de,https://www.upsidedown-studio.de/,+491724519252,https://www.eversports.de/s/upsidedown-studio-...,"[Yoga, Yoga, Yoga, Poledance, Krafttraining, 4...",1 Angebot für Neukund:innen,2024-02-03 16:57:28,2024-02-03 16:57:28
157,Seemannsbraut Poledance,"[Ochsenweg 27-29 , 24941 Flensburg]",24941,Flensburg,Ochsenweg 27-29,"[Übersicht, Klassen, Kurse, Workshops, Preise,...",Moin und willkommen an Bord in unserem Poledan...,seemannsbraut-poledance@web.de,http://www.seemannsbraut-poledance.de,None,https://www.eversports.de/s/seemannsbraut-pole...,"[Poledance, HIIT, Yoga, Poledance, Poledance, ...",1 Angebot für Neukund:innen,2024-02-03 16:57:36,2024-02-03 16:57:36
158,UpsideDown Studio Bad Homburg,"[Louisenstraße 98 , 61348 Bad Homburg vor der...",61348,Bad,Louisenstraße 98,"[Übersicht, Klassen, Kurse, Preise, Team]",Das Studio UpsideDown ist in Bad Homburg vor d...,info@upsidedown-studio.de,https://www.upsidedown-studio.de/,+491724519252,https://www.eversports.de/s/upsidedown-studio-...,"[Fitness, Tanzen, Yoga, Fitness, Tanzen, Yoga,...",1 Angebot für Neukund

In [6]:
result_workshop_details

,Workshop Name,Description,Studio Name,Location,Level,Date,URL_E,Time
0,Twerk it,Es kommt Schwung in die Kiste! Möchtet ihr eur...,Pole Faction - Darmstadt,"Twerk Ticket für So. 24.03.2024, 18:00 - 19:30",Choreographie,"Begann am Sonntag, 28. Januar 2024",https://www.eversports.de/e/workshop/FP-cEyn,"Heidelberger Landstrasse 237, 64297 Darmstadt"
1,ONLINE Heels Flow | Pole Workshop,Tauche ein in die sinnliche Welt des Pole Danc...,Les.Pole,Online,Poledance,"Samstag, 24. Februar 2024",https://www.eversports.de/e/workshop/ph8TlfO,12:00 - 13:30
2,ONLINE Perfect Pictures | Pole Workshop,"Bei dir steht ein Fotoshooting an, aber dir fe...",Les.Pole,Online,Poledance,"Freitag, 26. April 2024",https://www.eversports.de/e/workshop/4lKTG4R,19:00 - 20:30
3,KASSEL+ONLINE Chairdance Perfect Pictures Work...,"Bei dir steht ein Fotoshooting an, aber dir fe...",Les.Pole,Online,Poledance,"Samstag, 4. Mai 2024",https://www.eversports.de/e/workshop/lByHwWr,12:00 - 13:30
4,BREMEN Pole X Heels,"Musik an, jetzt wird getanzt!... Leg Swirls, B...",Les.Pole,Einzelticket,Poledance,"Samstag, 18. Mai 2024",https://www.eversports.de/e/workshop/VC_twiJ,15:00 - 16:30
...,...,...,...,...,...,...,...,...
287,Handstand,Du möchtest gerne den Handstand lernen und dir...,Pole and Move Ketsch,Einzelticket,Fitness,"Samstag, 2. März 2024",https://www.eversports.de/e/workshop/NCNYd8p,13:45 - 14:55
288,VALENTINS WORKSHOPS,Entdeckt im Februar die Magie der Liebe mit un...,Seemannsbraut Poledance,"Chairdance, 09.02.2024, 17.15 - 18.45 Uhr",Poledance,"Beginnt am Freitag, 9. Februar 2024",https://www.eversports.de/e/workshop/tjSmCEx,"Ochsenweg 27-29, 24941 Flensburg"
289,Lea Roth - Favorite Tricks & Combos,Favorite Tricks and Combos : 90 min Hier zeige...,Seemannsbraut Poledance,"Ochsenweg 27-29, 24941 Flensburg",Poledance,"Samstag, 17. Februar 2024",https://www.eversports.de/e/workshop/mQPzgO-,18:00 - 19:30
290,POLE NICK „Recent Favorite Tricks & Transition...,Nick zeigt dir seine Lieblingstricks und -Komb...,Seemannsbraut Poledance,"Ochsenweg 27-29, 24941 Flensburg",Poledance,"Freitag, 10. Mai 2024",https://www.eversports.de/e/workshop/ziaHmeW,13:00 - 14:30
